In [ ]:
import json
import sys
import os
from AnswerGeneration.getAnswer import generate_answer_withContext

def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def getRanking(file_path, query_id):
    ranking = load_json(file_path)
    # print(f"Ranking from file {file_path}: {ranking}")
    for rank in ranking:
        if query_id in rank:
            return rank[query_id][0]
    print("Query ID not found")
    return None

def getDocument(file_path, context_document_id):
    documents = load_json(file_path)
    
    found = 0
    context_document_text = {}
    # Iterate over documents and check if any document matches the wikipedia_id
    for doc in documents:
        # Iterate through context_document_id to find the corresponding key (query_id)
        for query_id, wiki_id in context_document_id.items():
            # Check if the wikipedia_id matches the value (wiki_id)
            if wiki_id == doc["wikipedia_id"]:
                # Update the query_id with the joined text of the document
                context_document_text[query_id] = " ".join(doc["text"])  
                found += 1

    print(f"Found {found} documents")
    return context_document_text

query_ids = load_json("QnA_Eval_Query_ids.json")
query = load_json("../Datasets/FinalDataset.json")


files = ["tf-idf_1_0.json"]

for file in files:
    results = []
    context_document_id = {}
    for query_id in query_ids:
        context_document_id[query_id] = getRanking(file, query_id)

    miniWikiCollection_path = "../Datasets/mini_wiki_collection.json"
    context_documents = getDocument(miniWikiCollection_path, context_document_id)

    for query_id in query_ids:
        context_text = context_documents[query_id]
        query_text = query[query_id]["input"]
        answer_text = list(query[query_id]["output"].values())[0]
        response = generate_answer_withContext(query_text, context_text)
        results.append({
            "query_id": query_id,
            "top_document_id": context_document_id[query_id],  
            "input" : query_text,
            "response": response,
            "gold_answer": answer_text
        })
        print(results)
    break
